In [103]:
import ftplib
from io import BytesIO

import pandas as pd

import regex

In [156]:
try:
    ftp = ftplib.FTP('ftp.ncdc.noaa.gov') 
    ftp.login()
    print('Connect successful')

except ftplib.all_errors as e:
    errorcode_string = str(e).split(None, 1)[0]

Connect successful


In [112]:
ftp.cwd("/pub/data/ghcn/daily")

'250 CWD command successful'

In [100]:
r = BytesIO()

ftp.retrbinary('RETR ghcnd-stations.txt', r.write)

r.seek(0)

stations = pd.read_fwf(r, index_col=False, error_bad_lines=False, encoding='utf-8', header=None)

In [101]:
stations.columns = ["id", "latitude", "longitude", "elevation", "name", "gsn_flag", "hcn_flag", "wmo_id"]

In [102]:
stations.head()

,id,latitude,longitude,elevation,name,gsn_flag,hcn_flag,wmo_id
0,ACW00011604,17.1167,-61.7833,10.1,ST JOHNS COOLIDGE FLD,NaN,NaN,NaN
1,ACW00011647,17.1333,-61.7833,19.2,ST JOHNS,NaN,NaN,NaN
2,AE000041196,25.3330,55.5170,34.0,SHARJAH INTER. AIRP,NaN,GSN,41196.0
3,AEM00041194,25.2550,55.3640,10.4,DUBAI INTL,NaN,NaN,41194.0
4,AEM00041217,24.4330,54.6510,26.8,ABU DHABI INTL,NaN,NaN,41217.0


In [95]:
r = BytesIO()

ftp.retrbinary('RETR ghcnd-countries.txt', r.write)

r.seek(0)

country_prefix = pd.read_fwf(r, index_col=False, error_bad_lines=False, encoding='utf-8', header=None)

In [96]:
country_prefix.columns = ["prefix", "name"]

In [97]:
country_prefix.head()

,prefix,name
0,AC,Antigua and Barbuda
1,AE,United Arab Emirates
2,AF,Afghanistan
3,AG,Algeria
4,AJ,Azerbaijan


In [98]:
country_prefix[country_prefix.name == "India"]

,prefix,name
89,IN,India


In [107]:
subset = stations.id.str.contains("^IN")

sum(subset)

3807

In [109]:
india = stations[subset]

In [157]:
ftp.cwd("/pub/data/ghcn/daily/all/")

'250 CWD command successful'

In [158]:
col_widths = [21]+[5, 1, 1, 1]*31

In [171]:
india.reset_index(drop=True, inplace=True)

In [172]:
india[india.id=="IN020083300"]

,id,latitude,longitude,elevation,name,gsn_flag,hcn_flag,wmo_id
3153,IN020083300,10.08,78.25,173.0,PULIPATTI,NaN,NaN,NaN


In [173]:
for station in india.id[3153:]:
    
    file = open("backup/"+station+".dly", 'wb')
    
    ftp.retrbinary('RETR {id}.dly'.format(id=station), file.write)
    
    file.close()

In [140]:
data_list = []

for file in glob.glob1("backup", "*.dly"):

    temp = pd.read_fwf(file, widths = col_widths, index_col=False, error_bad_lines=False, encoding='utf-8', header=None)
    
    data_list.append(temp)
    
weather = pd.concat(data_list)

KeyboardInterrupt: 

In [ ]:
weather.shape